In [1]:
import pandas as pd
import os, glob, re

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("../pbn_t3_5_v0.3.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Author: Luc Jonveaux
ID: 1 	 License: CC BY-NC-SA
ID: 2 	 Language: English
ID: 3 	 Project: PROBONO
ID: 4 	 Task: T3.5
ID: 5 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 6 	 Changes from: 0.0
ID: 7 	 Changes: Creation of the knowledge graph
ID: 8 	 Changes from: 0.1
ID: 9 	 Changes: Adding relations, simplified ontology, closest items, and descriptions
ID: 10 	 Repository: https://github.com/mm80843/T3.5/
ID: 11 	 TODOs: Quite a number! Creating metaRisks, making it accessible..
ID: 12 	 Creation: 11/11/2023
ID: 13 	 Version: 0.3
ID: 14 	 VersionComment: New blueprints added in 0.3
ID: 15 	 Library: owlready2==0.45


In [3]:
for a in onto.Article.instances():
    LABELS = a.label
    LABELS = sorted(LABELS, key=len)
    a.label = [LABELS[-1].capitalize()]

In [4]:
dIDct = u.createDict(onto) 

In [5]:
u.cOnto(onto)


# Overview

*  _PBNThing_  --  19260 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _PBNCategory_  --  330 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _aBlueprint_  --  50 instances.


In [6]:
def cleanTitle(x):
    x = str(x).replace("("," ").replace(")"," ")
    x = re.sub(' +', ' ',  str(x)).strip().capitalize()
    return x

In [7]:
DOIs =[]
for a in onto.Article.instances():
    if a.label:
        a.label = [cleanTitle(a.label[0]) ]
    if a.has_ArticleDOI:
        A = a.label
        A = sorted(A, key=len)
        DOIs.append([a.has_ArticleDOI,A[0]])
dfDOI = pd.DataFrame(DOIs,columns=["doi","TITLE"])
#dfDOI.TITLE = dfDOI.TITLE.apply(lambda x: cleanTitle(x))

dfDOI["LEN"] = dfDOI.TITLE.apply(lambda x: len(x))
dfDOI.sort_values(by="LEN").head(10)

,doi,TITLE,LEN
0,[10.1016/j.scitotenv.2020.141750],[Detection of sars-cov-2 in raw and treated wa...,1
247,[10.1016/j.eap.2022.03.012],[Covid-19 and regional solutions for mitigatin...,1
246,[10.1101/2020.05.10.20097329],[A continuously active antimicrobial coating e...,1
245,[10.31219/osf.io/rf7xa],[The impact of covid-19 on public space: a rev...,1
244,[10.21203/rs.3.rs-66806/v1],[Overcoming the impact of covid-19 using integ...,1
243,[10.1016/j.jum.2021.01.002],[Revisiting the built environment: 10 potentia...,1
242,[10.1136/bmj.m1066],[Covid-19 and community mitigation strategies ...,1
241,[10.3390/smartcities2010007],[The emergence of anti-privacy and control at ...,1
240,[10.1007/s42979-021-00923-y],[Pandemic analytics: how countries are leverag...,1
248,[10.1007/s43069-020-0014-9],[How architecture fails in conditions of crisi...,1


In [8]:
dfR = pd.read_parquet("newrisks.parquet.gzip")
dfR = dfR.replace("","")
dfR = dfR.merge(dfDOI,on="doi",how="left").sort_values(by="TITLE")
dfR

/home/kelu/projets/T3.5/venv/lib/python3.10/site-packages/pandas/core/missing.py:106: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  new_mask[arr_mask] = arr[arr_mask] == x


TypeError: unhashable type: 'IndividualValueList'

In [ ]:

for ix, row in dfR.iterrows():
    if ix < 20000:
        try:
            newRisk =  u.addItem("Risk", row["Description"], dIDct, onto )
            newRisk.has_RiskName.append(row["Impact"])

            src= u.addItem("Article",row["TITLE"], dIDct, onto )
            newRisk.has_RiskSource.append(src)

            mitig = u.addItem("Mitigation",row["Mitigation"], dIDct, onto )
            newRisk.has_RiskMitigation.append(mitig)
            mitig.has_MitigationSource.append(src)
            
            for tech in row["Technologies"]:
                mytech = u.addItem("Technology",tech, dIDct, onto ) 
                newRisk.has_RiskTechnology.append(mytech)
                mytech.has_TechSource.append(src)

            for stakeholder in row["People"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )  
                newRisk.has_RiskSubject.append(newstakeholder)
                                
            for stakeholder in row["Owner"]:
                newstakeholder = u.addItem("Stakeholder",stakeholder, dIDct, onto )   
                newRisk.has_RiskOwner.append(newstakeholder)
        except:
            pass


In [ ]:
DOIs =[]
for a in onto.Article.instances():
    DOIs.append([a.has_ArticleDOI,a.has_ArticleTitle,a.label[0]])
dfDOIs = pd.DataFrame(DOIs,columns=["doi","TITLE","titre"]).sort_values(by="titre")
dfDOIs

,doi,TITLE,titre
167,[10.1111/ciso.12282],[10 Adaptive Measures for Public Places to fac...,10 adaptive measures for public places to face...
188,[10.1128/mSystems.00245-20],[2019 Novel Coronavirus (COVID-19) Pandemic: B...,2019 novel coronavirus (covid-19) pandemic: bu...
376,[],[],2019 novel coronavirus covid-19 pandemic: buil...
318,[10.1016/j.ypmed.2021.106425],[A call to action: Improving urban green space...,A call to action: improving urban green spaces...
212,[10.1088/1757-899x/1055/1/012083],[A Comprehensive Review of Applications of Int...,A comprehensive review of applications of inte...
...,...,...,...
307,[10.1080/0960085X.2020.1770632],[What drives unverified information sharing an...,What drives unverified information sharing and...
163,[10.1787/65cfc31c-en],[What has been the impact of the COVID-19 pand...,What has been the impact of the covid-19 pande...
227,[10.1088/1748-9326/abb851],[What our response to the COVID-19 pandemic te...,What our response to the covid-19 pandemic tel...
4,[10.1016/j.scitotenv.2020.140980],[When the fourth water and digital revolution ...,When the fourth water and digital revolution e...


In [ ]:
u.cOnto(onto)


# Overview

*  _PBNThing_  --  86888 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  393 instances.
*  _Risk_  --  20899 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  16002 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  23093 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  22885 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _PBNCategory_  --  330 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _aBlueprint_  --  50 instances.


In [ ]:
onto.save("./WIP.owl")

In [ ]:
print("Done")

Done
